# Variable Importance Test

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath("../src"))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from BRAT.algorithms import BRATD

## Data Generating Process

In [ ]:
rng = np.random.default_rng(0)
n, d = 1000, 5
X = rng.uniform(0, 1, (n, d))
y = np.sin(2*np.pi*X[:,0]) + 0.5*X[:,1] + rng.normal(scale=1, size=n)

X_train, X_hold, y_train, y_hold = train_test_split(
    X, y, test_size=0.1, random_state=0
)
d_check = 2
X_train_sub = X_train[:, :d_check]
X_hold_sub  = X_hold[:,  :d_check]

## Model Fitting

In [ ]:
common = dict(
    n_estimators   = 100,
    learning_rate  = 1.0,
    max_depth      = 6,
    subsample_rate = 0.8,
    dropout_rate   = 0.8,
    disable_tqdm   = False,
)
model_full = BRATD(**common)
model_full.fit(X_train,    y_train, X_hold,    y_hold)

model_sub = BRATD(**common)
model_sub.fit(X_train_sub, y_train, X_hold_sub, y_hold)

## The Difference Vectors

In [ ]:
model_full.full_K()
model_sub .full_K()

m_pts = X_hold.shape[0]
n_train = X_train.shape[0]

R_full = np.zeros((m_pts, n_train))
R_sub  = np.zeros((m_pts, n_train))

for j, x in tqdm(enumerate(X_hold), desc = "Sketching R_full, R_sub", total=m_pts):
    rn_full, _ = model_full.sketch_r(x, vector=True)
    rn_sub,  _ = model_sub .sketch_r(x[:d_check], vector=True)
    R_full[j, :] = rn_full
    R_sub [j, :] = rn_sub

R_diff = R_full - R_sub  # shape: m*n

## Estimating Noise Variance

In [ ]:
full_sigma2 = model_full.est_sigma_hat2(in_bag=False)
sub_sigma2  = model_sub .est_sigma_hat2(in_bag=False)
sigma2 = (full_sigma2 + sub_sigma2) / 2

## VCV matrix calculation

In [ ]:
Sigma = sigma2 * (R_diff @ R_diff.T)
Sigma += 1e-8 * np.eye(m_pts)      # numerical ridge
Sigma_inv = np.linalg.inv(Sigma)

## Test Statistics

In [ ]:
delta = model_full.predict(X_hold) - model_sub.predict(X_hold_sub)  # m-vector
T = float(delta.T @ Sigma_inv @ delta)
p_val = 1 - chi2.cdf(T, df=m_pts)

print("==============================================")
print(f"Chi2 test (m = {m_pts} test points)")
print(f"  Test statistic     T = {T:.3f}")
print(f"  Degrees of freedom    = {m_pts}")
print(f"  sigma_hat2               = {sigma2:.4e}")
print(f"  p-value               = {p_val:.4g}")
print("  Decision (alpha=0.05):", "REJECT" if p_val < 0.05 else "FAIL TO REJECT")
print("==============================================")

# Type I, II Error

In [ ]:
rng = np.random.default_rng(66)
def run_test(n, weight_third=0.0, d=5, m_hold_pct=0.1, **common):
    """
    Runs the variable importance test for one dataset of size n.
    weight_third: signal weight for the third covariate (null=0, alternative>0).
    Returns p-value.
    """
    X = rng.uniform(0, 5, (n, d))
    # Generate y with optional signal in X[:,2]
    y = (4 * X[:,0]
        - X[:,1]**2
        + weight_third * (X[:,2])
        + rng.normal(scale=0.01, size=n)
        )

    
    # Train/hold split
    X_train, X_hold, y_train, y_hold = train_test_split(
        X, y, test_size=m_hold_pct, random_state=0
    )
    d_check = 2
    X_train_sub = X_train[:, :d_check]
    X_hold_sub  = X_hold[:,  :d_check]
    
    # Fit full and sub models
    model_full = BRATD(**common)
    model_full.fit(X_train, y_train, X_hold, y_hold)
    model_sub = BRATD(**common)
    model_sub.fit(X_train_sub, y_train, X_hold_sub, y_hold)
    
    # Build R matrices
    model_full.full_K()
    model_sub.full_K()
    m_pts = X_hold.shape[0]
    n_train = X_train.shape[0]
    
    R_full = np.zeros((m_pts, n_train))
    R_sub  = np.zeros((m_pts, n_train))
    for j, x in enumerate(X_hold):
        rn_full, _ = model_full.sketch_r(x, vector=True)
        rn_sub,  _ = model_sub.sketch_r(x[:d_check], vector=True)
        R_full[j, :] = rn_full
        R_sub [j, :] = rn_sub
    
    R_diff = R_full - R_sub
    full_sigma2 = model_full.est_sigma_hat2(in_bag=False)
    sigma2 = full_sigma2
    
    Sigma = sigma2 * (R_diff @ R_diff.T)
    Sigma_inv = np.linalg.pinv(Sigma)
    
    delta = model_full.predict(X_hold) - model_sub.predict(X_hold_sub)
    T = float(delta.T @ Sigma_inv @ delta)
    p_val = 1 - chi2.cdf(T, df=m_pts)
    return R_diff, sigma2, Sigma, Sigma_inv, T, p_val

In [ ]:
# Simulation settings
n_list = [20, 30, 50, 80, 100, 200]
train_n_list = [n * 0.9 for n in n_list]

reps = 30
common_params = dict(
    n_estimators   = 100,
    learning_rate  = 1.0,
    max_depth      = 8,
    subsample_rate = 1.0,
    dropout_rate   = 0.95,
    disable_tqdm   = True,
)

type1_errors = []
type2_errors = []

for n in n_list:
    results_null = [run_test(n, weight_third=0.0, **common_params) for _ in tqdm(range(reps), desc=f"Type I at n={n}")]
    results_alt  = [run_test(n, weight_third=5.0, **common_params) for _ in tqdm(range(reps), desc=f"Type II at n={n}")]
    R_diffs_null, sigma2s_null, Sigmas_null, Sigmas_inv_null, T_nulls, pvals_nulls = zip(*results_null)
    R_diffs_alt, sigma2s_alt, Sigmas_alt, Sigmas_inv_alt, T_alts, pvals_alts = zip(*results_alt)
    
    # summaries under null
    rdn = np.array(R_diffs_null)         # shape (reps, m_hold, n_train)
    s2n = np.array(sigma2s_null)         # shape (reps,)
    Tin = np.array(T_nulls)              # shape (reps,)

    print(f"R_diff under null:       {rdn.mean():.3f} ± {rdn.std():.3f}")
    print(f"sigma2 under null:       {s2n.mean():.3f} ± {s2n.std():.3f}")
    print(f"T-statistic under null:  {Tin.mean():.3f} ± {Tin.std():.3f}")

    # summaries under alt
    rda = np.array(R_diffs_alt)
    s2a = np.array(sigma2s_alt)
    Tia = np.array(T_alts)

    print(f"R_diff under alt:        {rda.mean():.3f} ± {rda.std():.3f}")
    print(f"sigma2 under alt:        {s2a.mean():.3f} ± {s2a.std():.3f}")
    print(f"T-statistic under alt:   {Tia.mean():.3f} ± {Tia.std():.3f}")

    # Type I error (false positive rate under null)
    type1_errors.append(np.mean(np.array(pvals_nulls) < 0.05))
    # Type II error (false negative rate under alternative)
    type2_errors.append(np.mean(np.array(pvals_alts) >= 0.05))
    print(f"Type I error at n={n}: {type1_errors[-1]:.3f}")
    print('Test statistic under null:', T_nulls)
    print(f"Type II error at n={n}: {type2_errors[-1]:.3f}")
    print('Test statistic under alternative:', T_alts)

out_dir = './variable_importance/'
os.makedirs(out_dir, exist_ok=True)

results = pd.DataFrame({
    'n':            train_n_list,
    'type_I_error': type1_errors,
    'type_II_error': type2_errors
})

results.to_csv(os.path.join(out_dir, 'size_and_power.csv'), index=False)
print(f"Saved summary to {os.path.join(out_dir, 'size_and_power.csv')}")

# Plotting
plt.figure(figsize=(2, 2))
plt.plot(train_n_list, type1_errors, marker='o', label='Type I Error')
plt.plot(train_n_list, type2_errors, marker='o', label='Type II Error')
plt.xlabel('Training sample size (n)')
plt.ylabel('Error rate')
plt.title('Type I and Type II Error Curves')
plt.legend()
plt.tight_layout()
plt.show()
